In [1]:
import sqlite3
import pandas as pd
import json
from progressbar import ProgressBar
from fastai.text import *

# Read Lablels

In [2]:
bs = 32

In [3]:
path = '../data/NELA2019/labels.csv'
labels_data = pd.read_csv(path)

In [4]:
labels_data.head()

,source,aggregated_label,"Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake",...,"Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stcenturywire,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,abcnews,0.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,...,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,activistpost,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,addictinginfo,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,adobochronicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
with open('../data/NELA2019/source-metadata.json') as file:
    metadata = (json.loads(file.read()))

In [6]:
for source in metadata.keys():
    labels_data.loc[labels_data.source == source, 'n_articles'] = metadata[source]['articles']

In [7]:
cols = ['source', 'n_articles', 'aggregated_label', 'Media Bias / Fact Check, label']
labels_data[cols].head(10)

,source,n_articles,aggregated_label,"Media Bias / Fact Check, label"
0,21stcenturywire,791.0,2.0,conspiracy_pseudoscience
1,abcnews,2861.0,0.0,left_center_bias
2,activistpost,2780.0,2.0,conspiracy_pseudoscience
3,addictinginfo,12.0,1.0,left_bias
4,adobochronicles,417.0,NaN,satire
5,ageofautism,646.0,2.0,conspiracy_pseudoscience
6,airwars,16.0,0.0,least_biased
7,aljazeera,11611.0,0.0,left_center_bias
8,americablognews,104.0,0.0,left_bias
9,anonnews,508.0,NaN,NaN


# Read Data

In [8]:
# Execute query and load results into pandas dataframe
def execute_query_pandas(path, query):
    conn = sqlite3.connect(path)
    df = pd.read_sql_query(query, conn)
    return df

In [80]:
path = '../data/NELA2019/nela-eng-2019.db'
#query= ''' SELECT * FROM newsdata WHERE strftime('%m', date) in ('01')'''
query= ''' SELECT * FROM newsdata '''
#query = ''' SELECT * FROM newsdata WHERE source='21stcenturywire' '''

data = execute_query_pandas(path, query)

In [81]:
for source in labels_data['source']:
    data.loc[data.source == source, 'label'] = labels_data.loc[labels_data.source == source, 'aggregated_label'].values[0]
    data.loc[data.source == source, 'bias'] = labels_data.loc[labels_data.source == source, 'Media Bias / Fact Check, label'].values[0]

data = data.dropna(subset = ['label', 'bias'], how='any')

In [82]:
data.sample(3)

,id,date,source,title,content,author,url,published,published_utc,collection_utc,label,bias
196192,bbcuk--2019-01-13--Gambling addiction Flaws ex...,2019-01-13,bbcuk,Gambling addiction: Flaws exposed in online se...,The head of a scheme designed to help problem ...,,https://www.bbc.co.uk/news/uk-46830810,2019-01-13 11:11:05+00:00,1547395865,1567552714,0.0,left_center_bias
224187,newyorker--2019-08-28--Daily Cartoon Wednesday...,2019-08-28,newyorker,"Daily Cartoon: Wednesday, August 28th",How to Write a New Yorker Cartoon Caption: Kum...,Emily Flake,https://www.newyorker.com/cartoons/daily-carto...,2019-08-28 14:57:41+00:00,1567018661,1567543637,1.0,left_bias
151766,shareblue--2019-04-10--GOP leader wont rule ou...,2019-04-10,shareblue,GOP leader won't rule out supporting ex-convic...,Former New York Republican Rep. Michael Grimm ...,Emily Singer,https://shareblue.com/gop-leader-kevin-mccarth...,2019-04-10 22:00:28+00:00,1554948028,1567543385,0.0,left_bias


# Process Data

We tokenize and build the vocabulary

In [83]:
data_lm = (TextList.from_df(data, cols='content')
                                .split_by_rand_pct(0.1)
                                .label_from_df(cols='label')
                                .label_for_lm() 
                                .databunch(bs=32))

In [84]:
data_lm.save('data_lm.pkl')

In [94]:
#data_lm.show_batch()
#data_lm.vocab.itos[30:40]
#data_lm.train_ds[1][0]
#data_lm.train_ds[0][0].data[:10]

In [25]:
data_lm = load_data('.', 'data_lm.pkl', bs=bs)

In [87]:
#data_lm.show_batch()

In [96]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, 
                                metrics=[accuracy, Perplexity()])

In [97]:
TEXT = "The new book"
N_WORDS = 40
N_SENTENCES = 2
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

The new book The Adventures of Macdonald and Millimeters , published in Australia , 1998 , and Donald Sutherland and the Rite of Spring , The Adventures of Macleod , was
The new book The Times of America was written by Robert Ellis Murray and Edwin Brick Anderson . They published a short book about the role of the American Civil


In [100]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,perplexity,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [99]:
learn.recorder.plot(suggestion=True)

In [102]:
learn.fit_one_cycle(4, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: 